In [1]:
# LTN: https://github.com/logictensornetworks/logictensornetworks/blob/master/examples/multiclass_classification/multiclass-singlelabel.ipynb
# Common.py : https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py

In [2]:
!pip install PyTDC rdkit-pypi ltn keras==2.15.0 -qq
!wget https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.3/151.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/4

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import logging; logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import pandas as pd
import numpy as np
# import ltn
# from rdkit import Chem
# from rdkit.Chem import AllChem
from tqdm.auto import tqdm
tqdm.pandas()

In [7]:
import torch
from transformers import AutoTokenizer, AutoModel,pipeline

In [ ]:
from huggingface_hub import login

hf_token = "<your_hf_secret_code>"
login(token=hf_token, add_to_git_credential=True)

In [10]:
# Threshold ref: https://pubs.acs.org/doi/epdf/10.1021/acs.jcim.3c01301
def label_th(pic50):
    classes = []
    for x in pic50:
        if x>=5:
            classes.append(1)
        else:
            classes.append(0)

    return np.asarray(classes)
class_map = {
    "blocks":1,
    "non-blocks":0,
}

### Data Acquisition

In [19]:
dataset_path = "/content/drive/MyDrive/Project/AI and Cardiology/Cardiotoxicity/Dataset"

In [20]:
!ls "{dataset_path}/UniChemDB-Data"

CDK-unichemdb.csv     herg-gemini-embedding.parquet  standized-herg.csv
final-herg.csv	      herg_uniherg_db_mmb_emb.npy    uniherg_db-deepseek-qwen1_5b-embedding.parquet
final-herg-split.csv  Morgan-unichemdb.csv


In [21]:
df = pd.read_csv(f"{dataset_path}/UniChemDB-Data/final-herg-split.csv")
df.dropna(subset = ['std_smiles'],inplace = True)
df.reset_index(drop = True,inplace = True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20409 entries, 0 to 20408
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                20388 non-null  object
 1   std_smiles        20409 non-null  object
 2   classes           20409 non-null  int64 
 3   train_test_split  20409 non-null  int64 
 4   cv_fold           20409 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 797.4+ KB


# Data



In [23]:
#External Test-1: https://github.com/Abdulk084/CardioTox/blob/master/data/external_test_set_pos.csv
ext_pos_df = pd.read_csv(f"{dataset_path}/External-Data/external_test_set_pos.csv")

In [24]:
# External Test h70, h60 dataset: https://github.com/issararab/CToxPred/tree/main/data/raw/hERG
ext_h60_df = pd.read_csv(f"{dataset_path}/External-Data/eval_set_herg_60.csv")
ext_h70_df = pd.read_csv(f"{dataset_path}/External-Data/eval_set_herg_70.csv")


In [25]:
ext_h60_df.head()

,InChl Key,SMILES,Source,pIC50
0,LIHJHFVXLZSRNK-UHFFFAOYSA-N,Cn1ccc(C[N+]2=CC(c3cccc(C(F)(F)F)c3)C=N2)n1,US Patent,5.647817
1,RXGDDWPITVSKDR-UHFFFAOYSA-N,CC(C)(C)OC(=O)N1CCN(c2nc3c([N+](=O)[O-])c(Br)c...,US Patent,5.602060
2,YRSBMPKJFDYYFO-UHFFFAOYSA-N,Fc1cccc(Oc2cc(C(F)(F)F)nc(N3CCc4nc[nH]c4C3)n2)c1,US Patent,5.599980
3,OMQQLDITRYIEHZ-UHFFFAOYSA-N,Cn1nccc1Cc1cn(-c2ccc(F)c(Cl)c2)nn1,US Patent,5.364516
4,BXBUTKPGTGJGTQ-YOEHRIQHSA-N,CNC[C@@H](c1ccc(Cl)c(Cl)c1)[C@@H](OC)c1cccc(NS...,US Patent,5.327902


In [15]:
# Load LLaMA-3 tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-1B')  # Replace with the correct model ID
model = AutoModel.from_pretrained('meta-llama/Llama-3.2-1B')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

In [16]:
## HF Pipeline for feature extraction
device= 'cuda' if torch.cuda.is_available() else 'cpu'
pipe =pipeline('feature-extraction',model= model,tokenizer = tokenizer,device_map= 'auto',device = device)

Both `device` and `device_map` are specified. `device` will override `device_map`. You will most likely encounter unexpected behavior. Please remove `device` and keep `device_map`.
Device set to use cuda


In [17]:
!ls '{dataset_path}/External-Data'

CDK-external_test_set_pos.csv
CDK-herg60.csv
CDK-herg70.csv
eval_set_herg_60.csv
eval_set_herg_70.csv
external_test_set_pos.csv
h60-uniherg_db-deepseek-qwen1_5b-embedding.parquet
h60-uniherg_db-gemini-embedding.parquet
h70-uniherg_db-deepseek-qwen1_5b-embedding.parquet
h70-uniherg_db-gemini-embedding.parquet
herg_mmb_emb_external_test.npz
herg_mmb_emb_h60.npz
herg_mmb_emb_h70.npz
Morgan-external_test_set_pos.csv
Morgan-herg60.csv
Morgan-herg70.csv
pos-uniherg_db-deepseek-qwen1_5b-embedding.parquet
pos-uniherg_db-gemini-embedding.parquet


In [26]:

# Function to compute embeddings for a single SMILES string
def compute_embedding(smiles):
    embeddings = pipe(smiles,return_tensors = 'pt').mean(dim=1).detach().cpu().numpy()[0]
    return embeddings

# Compute embeddings for all SMILES strings in the DataFrame
df['Embeddings'] = df['std_smiles'].progress_apply(lambda x: compute_embedding(x))
ext_h60_df['emb'] = ext_h60_df['SMILES'].progress_apply(lambda x: compute_embedding(x))
ext_h70_df['emb'] = ext_h70_df['SMILES'].progress_apply(lambda x: compute_embedding(x))
ext_pos_df['emb'] = ext_pos_df['smiles'].progress_apply(lambda x: compute_embedding(x))

  0%|          | 0/20409 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/473 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

In [27]:
import os
ext_h70_df.to_parquet(os.path.join(dataset_path,'External-Data','h70-uniherg_db-llama-embedding.parquet'),index = False)
ext_pos_df.to_parquet(os.path.join(dataset_path,'External-Data','pos-uniherg_db-llama-embedding.parquet'),index = False)
ext_h60_df.to_parquet(os.path.join(dataset_path,'External-Data','h60-uniherg_db-llama-embedding.parquet'),index = False)

In [28]:
df.to_parquet(os.path.join(dataset_path,'UniChemDB-Data','herg-llama-embedding.parquet'),index = False)

In [21]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X = ss.fit_transform(np.vstack(df['Embeddings'].values) )
y = df['classes']


In [22]:
X.shape,y.shape

((20409, 2048), (20409,))

In [23]:
batch_size = 64
ds_train = tf.data.Dataset.from_tensor_slices((X,y)).batch(batch_size)
idx = np.random.random_integers(0,len(X),1000)
ds_test = tf.data.Dataset.from_tensor_slices((X[idx],y[idx])).batch(batch_size)

<ipython-input-23-21e067a5c8e3>:3: DeprecationWarning: This function is deprecated. Please call randint(0, 20409 + 1) instead
  idx = np.random.random_integers(0,len(X),1000)


# LTN

Predicate with softmax `P(x,class)`

In [24]:
class MLP(tf.keras.Model):
    """Model that returns logits."""
    def __init__(self, n_classes, hidden_layer_sizes=(16,16,8)):
        super(MLP, self).__init__()
        self.denses = [tf.keras.layers.Dense(s, activation="elu") for s in hidden_layer_sizes]
        self.dense_class = tf.keras.layers.Dense(n_classes)
        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, inputs, training=False):
        x = inputs[0]
        for dense in self.denses:
            x = dense(x)
            x = self.dropout(x, training=training)
        return self.dense_class(x)

logits_model = MLP(2)
p = ltn.Predicate.FromLogits(logits_model, activation_function="softmax", with_class_indexing=True)

Constants to index/iterate on the classes

In [25]:
class_A = ltn.Constant(0, trainable=False)
class_B = ltn.Constant(1, trainable=False)
# class_C = ltn.Constant(2, trainable=False)

Operators and axioms

In [26]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")

In [27]:
formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))

@tf.function
def axioms(features, labels, training=False):
    x_A = ltn.Variable("x_A",features[labels==0])
    x_B = ltn.Variable("x_B",features[labels==1])
    # x_C = ltn.Variable("x_C",features[labels==2])
    axioms = [
        Forall(x_A,p([x_A,class_A],training=training)),
        Forall(x_B,p([x_B,class_B],training=training)),
        # Forall(x_C,p([x_C,class_C],training=training))
    ]
    for i in range(len(axioms)):
        if tf.math.is_nan(axioms[i].tensor):
            axioms[i].tensor  =0.0
    sat_level = formula_aggregator(axioms).tensor
    return sat_level

Initialize all layers and the static graph

In [28]:
for features, labels in ds_test:
    print("Initial sat level %.5f"%axioms(features,labels))
    break

Initial sat level 0.45521


# Training

Define the metrics. While training, we measure:
1. The level of satisfiability of the Knowledge Base of the training data.
1. The level of satisfiability of the Knowledge Base of the test data.
3. The training accuracy.
4. The test accuracy.

In [29]:
metrics_dict = {
    'train_sat_kb': tf.keras.metrics.Mean(name='train_sat_kb'),
    'test_sat_kb': tf.keras.metrics.Mean(name='test_sat_kb'),
    'train_accuracy': tf.keras.metrics.CategoricalAccuracy(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
}

Define the training and test step

In [30]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
@tf.function
def train_step(features, labels):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(features, labels, training=True)
        loss = 1.-sat
    gradients = tape.gradient(loss, p.trainable_variables)
    optimizer.apply_gradients(zip(gradients, p.trainable_variables))
    sat = axioms(features, labels) # compute sat without dropout
    metrics_dict['train_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['train_accuracy'](tf.one_hot(labels,2),predictions)

@tf.function
def test_step(features, labels):
    # sat
    sat = axioms(features, labels)
    metrics_dict['test_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['test_accuracy'](tf.one_hot(labels,2),predictions)

Train

In [31]:
import commons

EPOCHS = 500

commons.train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    train_step,
    test_step,
    csv_path="herg_LLama_results.csv",
    track_metrics=20
)

Epoch 0, train_sat_kb: 0.4698, test_sat_kb: 0.5295, train_accuracy: 0.5945, test_accuracy: 0.6630
Epoch 20, train_sat_kb: 0.5682, test_sat_kb: 0.6073, train_accuracy: 0.7683, test_accuracy: 0.7720
Epoch 40, train_sat_kb: 0.6054, test_sat_kb: 0.6368, train_accuracy: 0.8186, test_accuracy: 0.8190
Epoch 60, train_sat_kb: 0.6290, test_sat_kb: 0.6542, train_accuracy: 0.8445, test_accuracy: 0.8280
Epoch 80, train_sat_kb: 0.6419, test_sat_kb: 0.6732, train_accuracy: 0.8572, test_accuracy: 0.8590
Epoch 100, train_sat_kb: 0.6597, test_sat_kb: 0.6803, train_accuracy: 0.8730, test_accuracy: 0.8610
Epoch 120, train_sat_kb: 0.6702, test_sat_kb: 0.6902, train_accuracy: 0.8819, test_accuracy: 0.8770
Epoch 140, train_sat_kb: 0.6815, test_sat_kb: 0.7015, train_accuracy: 0.8909, test_accuracy: 0.8810
Epoch 160, train_sat_kb: 0.6857, test_sat_kb: 0.7096, train_accuracy: 0.8951, test_accuracy: 0.8920
Epoch 180, train_sat_kb: 0.6946, test_sat_kb: 0.7096, train_accuracy: 0.9010, test_accuracy: 0.8930
Epoch 

In [34]:
!ls "{dataset_path}/../Model-Weights"

hERG-Karim-CDK.keras	     hERG-Karim-Morgan.keras	 hERG-UniChemDB-MMB.keras
hERG-Karim-MMB.keras	     hERG-UniChemDB-CDK.keras	 hERG-UniChemDB-Morgan_CDK.keras
hERG-Karim-Morgan_CDK.keras  hERG-UniChemDB-LLAMA.keras  hERG-UniChemDB-Morgan.keras


In [33]:
logits_model.save(f"{dataset_path}/../Model-Weights/hERG-UniChemDB-LLAMA.keras")

## Model Evaluation

In [35]:
from sklearn.metrics import (
    accuracy_score as ays,
    f1_score as fs,
    precision_score as ps,
    recall_score as rs,
    matthews_corrcoef as mcc,
    roc_auc_score as auc,
    balanced_accuracy_score,
    confusion_matrix
)

In [36]:
def print_score(xtest,ytest,name):

    pred_test = logits_model.predict([xtest]).argmax(-1)

    auc_test = auc(ytest, pred_test)


    tn, fp, fn, tp = confusion_matrix(ytest, pred_test).ravel()

    specificity_test = tn / (tn + fp)

    sensitivity_test = tp / (tp + fn)

    NPV_test = tn / (tn + fn)

    PPV_test = tp / (tp + fp)
    Accuracy_test = ays(ytest, pred_test)
    Balanced_Accuracy_test = balanced_accuracy_score(ytest, pred_test)

    MCC_test= mcc(ytest, pred_test)


    print(f"MCC_test_{name}: " + str(MCC_test))
    print(f"NPV_test_{name}g: " + str(NPV_test))
    print(f"Accuracy_test_{name}: " + str(Accuracy_test))
    print(f"PPV_test_{name}: " + str(PPV_test))
    print(f"specificity_test_{name}: " + str(specificity_test))
    print(f"sensitivity_test_{name}: " + str(sensitivity_test))
    print(f"Balanced_Accuracy_test{name}: " + str(Balanced_Accuracy_test))


In [37]:
print_score(ss.transform(np.vstack(ext_pos_df['emb'].values)),ext_pos_df['ACTIVITY'],'External Data Test-1 (pos)')


2/2 [==============================] - 0s 4ms/step
MCC_test_External Data Test-1 (pos): 0.660491160806468
NPV_test_External Data Test-1 (pos)g: 0.7058823529411765
Accuracy_test_External Data Test-1 (pos): 0.8409090909090909
PPV_test_External Data Test-1 (pos): 0.9259259259259259
specificity_test_External Data Test-1 (pos): 0.8571428571428571
sensitivity_test_External Data Test-1 (pos): 0.8333333333333334
Balanced_Accuracy_testExternal Data Test-1 (pos): 0.8452380952380952


In [38]:
print_score(ss.transform(np.vstack(ext_h60_df['emb'].values)),(ext_h60_df.pIC50 >=5).astype(int),'External hERG-60')


8/8 [==============================] - 0s 2ms/step
MCC_test_External hERG-60: 0.6206831605431194
NPV_test_External hERG-60g: 0.7388535031847133
Accuracy_test_External hERG-60: 0.8
PPV_test_External hERG-60: 0.9032258064516129
specificity_test_External hERG-60: 0.928
sensitivity_test_External hERG-60: 0.672
Balanced_Accuracy_testExternal hERG-60: 0.8


In [39]:
print_score(ss.transform(np.vstack(ext_h70_df['emb'].values)),(ext_h70_df.pIC50 >=5).astype(int),'External hERG-70')


15/15 [==============================] - 0s 6ms/step
MCC_test_External hERG-70: 0.5357484560765747
NPV_test_External hERG-70g: 0.6963562753036437
Accuracy_test_External hERG-70: 0.7632135306553911
PPV_test_External hERG-70: 0.8362831858407079
specificity_test_External hERG-70: 0.8229665071770335
sensitivity_test_External hERG-70: 0.7159090909090909
Balanced_Accuracy_testExternal hERG-70: 0.7694377990430622
